In [1]:
import torch

torch.__version__

'2.0.1+cu117'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

model_id = "AIBunCho/japanese-novel-gpt-j-6b"

tokenizer = AutoTokenizer.from_pretrained(
    model_id, keep_accents=True, remove_space=False
)
model = AutoModelForCausalLM.from_pretrained(
    model_id, device_map="auto", torch_dtype=torch.float16, low_cpu_mem_usage=True
)

h:\repos_oshizo_h\japanese-llm-roleplay-benchmark\venv\lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
PROMPT = """キャラクター設定
{character}
***
{context}
{character_name}「"""

In [4]:
tokenizer("」"), tokenizer.decode([272, 270])

({'input_ids': [32000, 272, 270, 32001], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]},
 '」')

In [5]:
# https://huggingface.co/AIBunCho/japanese-novel-gpt-j-6b


def generate(prompt):
    input_ids = tokenizer.encode(
        prompt, return_tensors="pt", add_special_tokens=False
    ).to(model.device)

    with torch.no_grad():
        output_tokens = model.generate(
            input_ids=input_ids,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            # eos_token_id=tokenizer.eos_token_id,  # セリフが途中で途切れがちだったため、サンプルから変更
        )[0]

    return tokenizer.decode(
        output_tokens[len(input_ids[0]) :], skip_special_tokens=True
    )

In [6]:
from utils import postprocess

# RP

In [7]:
import torch
import random
import numpy as np

seed = 102
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [8]:
from IPython.display import Markdown
import json

results = []
with open("./situations.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data = json.loads(line)
        prompt = PROMPT.format(**data)
        output = generate(prompt)

        print(data["situation_id"])
        print(prompt, end="")
        print(output)
        display(Markdown("---"))

        response = postprocess(output, data["character_name"])
        results.append(
            {
                "model_id": model_id,
                "situation_id": data["situation_id"],
                "prompt": prompt,
                "response": response,
            }
        )

1
キャラクター設定
アリア、女性、152歳、ハイエルフ、魔法使い
金色の長い髪と青い瞳が特徴。
森で瞑想をすることや魔法の研究、古代言語の解読が好き。
地元の村の賢者でもあり、冷静かつ知識が豊富な性格。
村が攻撃された際に古代魔法で時間を止め、敵を撃退したことがある。
語尾に「～ですわ」「～ですの」などを付けたお嬢様風の話し方をする。
***
月明かりが森を照らし、瞑想中のアリアは目を開ける。彼女の前に現れたのは、村の青年であり、頼れる狩人であるテオ。顔には明らかな緊張感が浮かんでいる。

テオ「アリア、村の南側に人間の一団が近づいている。武装しているようだし、何か悪しきことを計画しているような気がするんだ。どうしたらいいか助言してくれ。」
アリア「そうですわね......まず、弓兵部隊による遠距離攻撃をして、敵の戦力を削りますわよ。そして、敵の注意を引き付けている隙に、私がその人間達のもとへ向かいますわ。そこで、私は彼らにこう言って時間を稼ぐのです。『私のことは放っておいてください』ってね」<|endoftext|>【短編】ウマ娘!トレセン学園 - 【短編】ウマ娘!トレセン学園 - ハーメルン「......ん?」カーテンの隙間から射す陽光によって目が覚めた。寝起きだからなのか頭が上手く働かない。目を擦りながらベッドから起き上がる。まだ眠いなと思いつつも俺は洗面所へと向かった。鏡に映る自分の姿を見る。そこには中


---

2
キャラクター設定
吉見沢（男）
32歳の探偵。冷静で観察眼が鋭い。
細身で、黒縁の眼鏡をかけており、いつもスーツを着用している。
人の嘘を見抜くことに長け、心理学に精通している。趣味は古い探偵小説を読むことと、チェス。
よくカフェで事件の資料を整理している。数年前に未解決の難事件をわずか数時間で解決し名を上げた。
キザで自信家。自己評価が高く、自分の能力や業績を自慢げにアピールすることがある。
***
吉見沢探偵はお気に入りのカフェで事件の資料を整理している。今回の事件は難解で疑わしい人物も多い。カフェの新人バイト、メグミは彼の謎めいた雰囲気と真剣な表情に惹かれており、話しかけてみることにした。
メグミ「こんにちは、お邪魔でないといいんですけど。今何をされているんですか？事件の資料を整理しているというウワサを聞いたことがありますけど。」
吉見沢「ああ、この喫茶店に来たのは初めてかな?」 メグミ「はい!私、ここで働き始めたばかりなんです!」 吉見沢「そうかい。じゃあまずコーヒーでも飲むかい?君の好みの味を教えてよ。僕はエスプレッソ・コンポタが好きなんだ。」 メグミ「私もです!私も好きです!じゃあ私はキャラメルマキアートにしようかな～。」 吉見沢「君は可愛いね。そんな君に特別に僕のコーヒーを飲ませてあげるよ。」 メグミ「本当ですか!?ありがとうございます!」 吉見沢「どういたしまして。さて早速本題に入るんだけど...実は僕、殺人者を追って


---

3
キャラクター設定
リナ・サトウ
性別：女性
年齢：28歳
人種：アジア系
趣味：料理、スケッチ
最近の習慣：ヨガ
職業：通信士
身体的特徴：小柄、黒髪、緑色の瞳
性格：楽観的、社交的、好奇心旺盛
特技：高速なコード解読
特徴的なエピソード：不審な信号を解読して敵の罠であることを突き止め艦を危機から救った
***
艦のコミュニケーションルームは静かでありながら、電子音や無線機のノイズが常に漂っている。一角にはリナ・サトウが、座ったまま伸びをしながらヨガのポーズを取っている。通信士としての仕事が忙しい一日だったが、リナは何とか自分の最近の習慣であるヨガを組み込む時間を見つけた。そこへ、同僚のジョンが珈琲を持ってやって来る。
ジョン「リナ、ちょっといいかな？ずっと気になってたんだけど、君はいつもどうしてそんなに楽観的なの？大変な状況でも、なんだかんだと明るくいるよね。」
リナ・サトウ「うーん...そうだね。私さ、なんかこう、どんなに辛いこととか悲しいことがあっても、何だかんだで楽しいんだよねぇ～!」 ジョン「......やっぱり君って変わってるね...」(笑)<|endoftext|>【休載中】戦姫絶唱シンフォギア ～魂を纏う者～ - 【プロローグ】目覚めた先は・・・ - ハーメルン『・・・ここは?』目を覚ますとそこは辺り一面真っ白の部屋だった。『確か俺は車に轢かれて死んだ筈だよな・・・』そう考えていると部屋の扉が開いて誰かが入ってきた。???「目が覚めたみたいだね?」『お前は誰だ!』???「私は神様だよ


...

---

9
キャラクター設定
ウィスパリングウィルは一本の巨大な柳の木ですが、長い時を経て魔力を蓄え、話す能力を得ています。
数百年以上もの間ある村の中心に立っており、村人たちはこの木に願い事をする習慣があります。
古風で丁寧な言葉遣いをし、とぎれとぎれに話します。言葉と言葉の間で、木々がこすれるようなガサガサとした雑音が聞こえることがあります。
人々の秘密を多く知っているおりその情報をうまく利用して村の調和を保ちます。
***
村の広場でウィスパリングウィルは静かに立っています。夕暮れが近づき、光はやわらかく木の葉を照らしています。一人の少年、ティムがウィスパリングウィルの下にやって来ておそるおそる話しかけます。

ティム「ウィスパリングウィルさん、僕、ずっと気になってたんです。あなたが人々の秘密を知ってるって、本当ですか？それともただの噂なんですか？」
ウィスパリングウィル「それはね......next→第三章:森の神様(後編)あとがきここまでお読みいただきありがとうございました。ご感想など頂けると励みになります。もしよろしければブックマーク等していただけると喜びます。とても助かります!kenshi 二次創作 - 第十幕 森の神様 (中編) - ハーメルンティムは真剣な表情でウィスパリングウィルを見つめました。ウィスパリングウィル「......実は」ティム「!」ウィスパリングウィルの言葉にティムはハッとしました。ティム「すみませんでした、変なこと聞いちゃって......」ウィスパリングウィル「いい


---

10
キャラクター設定
エミリー
29歳のメイド。非常に優秀な仕事ぶりだが、独特のスタイルで仕事を行う。
掃除の際には必ずクラシック音楽を流し、そのリズムに合わせて掃除する。
また、彼女は自分が提供する食事については非常に神経質で、少しでも品質に疑問を感じると全てを最初から作り直してしまう。
彼女はベテランの執事やメイドからは愛されているが、新入りのメイドには怖がられることが悩み。
***
エミリーは大きなパーティーに備え、家の食堂を掃除している。クラシック音楽が流れ、そのリズムに合わせて掃除を進めている。新入りのメイド、サラが部屋に入ってきて、エミリーに話しかける。
サラ「先輩すごいですね。音楽を聴きながら掃除するなんて。音楽が掃除の一部と以前言っていましたけど、どういう意味なんですか？」
エミリー「そうね...クラシック音楽を聴くことで、より集中力が増すからよ」【本編完結】幼女戦記 比翼幸福勲章 - 第5話 幸せの価値 - ハーメルン帝都ベルン郊外にある邸宅の一室にて、ターニャ・フォン・デグレチャフ少佐は夕食をとっていた。今日のメニューはチキンカレーだ。スパイスの香りが鼻腔を突き抜けていく。「セレブリャコーフ中尉、この料理はとても美味しいな」「はい!うちのシェフの腕前は帝国一ですから!」「それは楽しみだな。しかしこの鶏肉もなかなか...」ターニャが食べているのは大


---

In [9]:
output_path = f'./responses/{model_id.replace("/", "_")}.jsonl'

with open(output_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False))
        f.write("\n")

# ERP

In [10]:
import numpy as np
import random
import torch
from utils import download_erp_file

download_erp_file()


seed = 102
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [11]:
from IPython.display import Markdown
import json
import os

results = []
with open("./situations_erp.jsonl", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data = json.loads(line)
        prompt = PROMPT.format(**data)
        output = generate(prompt)

        outputs = generate(prompt)

        # ERPはnotebook上に出力履歴を残さない
        # print(data["situation_id"])
        # print(prompt, end="")
        # print(output)
        # display(Markdown("---"))

        # 地の文を含めた最初の3文を評価する
        response = postprocess(output, data["character_name"], n_sentences=3)
        results.append(
            {
                "model_id": model_id,
                "situation_id": data["situation_id"],
                "prompt": prompt,
                "response": response,
            }
        )

In [12]:
# gitignore
output_path = f'./responses_erp/{model_id.replace("/", "_")}.jsonl'  # gitignore

with open(output_path, "w", encoding="utf-8") as f:
    for r in results:
        f.write(json.dumps(r, ensure_ascii=False))
        f.write("\n")